# Further Pre-training MobileBERT MLM with Federated Averaging (Stackoverflow)

In [1]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Google Colab settings

In [2]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    # Mount Google Drive root directory
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/Colab Notebooks/BERTerated'
    
    # List the directory contents
    !ls

### CUDA Multi GPU

In [3]:
# Use this code snippet to use specific GPUs
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

In [4]:
# IPython reloading magic
%load_ext autoreload
%autoreload 2

In [5]:
# Install required packages
# !pip install -r requirements.txt

## Import packages

In [6]:
import tensorflow as tf

In [7]:
tf_physical_devices_gpu = tf.config.list_physical_devices('GPU')

# Allow the growth of GPU memory consumption to take place incrementally
if tf_physical_devices_gpu:
    for gpu in tf_physical_devices_gpu:
        tf.config.experimental.set_memory_growth(gpu, True)

In [8]:
import os
import sys
import random
import datetime
import json
import pathlib
import itertools
import time

import numpy as np
import tensorflow_federated as tff
import tensorflow_text as tf_text
import tensorflow_addons as tfa
import transformers

import nest_asyncio
nest_asyncio.apply()

import fedavg
import fedavg_client
import datasets
import utils


# Random seed settings
random_seed = 692
random.seed(random_seed) # Python
np.random.seed(random_seed) # NumPy
tf.random.set_seed(random_seed) # TensorFlow

# Test if TFF is working
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [9]:
# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("TensorFlow version: " + tf.__version__)
print("TensorFlow Federated version: " + tff.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
NumPy version: 1.18.4
TensorFlow version: 2.3.1
TensorFlow Federated version: 0.17.0
Transformers version: 3.4.0


In [10]:
!nvidia-smi

Sat Nov 28 06:56:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    On   | 00000000:03:00.0 Off |                  N/A |
|  0%   36C    P2    46W / 205W |    231MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [11]:
tf_logical_devices_cpu = tf.config.list_logical_devices('CPU')
tf_logical_devices_gpu = tf.config.list_logical_devices('GPU')

## Experiment Settings

In [12]:
EXPERIMENT_CONFIG = {}

EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'] = 'google/mobilebert-uncased'
EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'] = os.path.join('.', 'transformers_cache')

EXPERIMENT_CONFIG['TOTAL_ROUNDS'] = 50 # Number of total training rounds
EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] = 1 # How often to evaluate

EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'] = 10 # How many clients to sample per round.
EXPERIMENT_CONFIG['TEST_CLIENTS_PER_ROUND'] = 10
EXPERIMENT_CONFIG['CLIENT_EPOCHS_PER_ROUND'] = 3

EXPERIMENT_CONFIG['BATCH_SIZE'] = 16 # Batch size used on the client.
EXPERIMENT_CONFIG['TEST_BATCH_SIZE'] = 16 # Minibatch size of test data.

# Maximum length of input token sequence for BERT.
EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'] = 128

# Optimizer configuration
EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'] = 1.0 # Server learning rate.
EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'] = 5e-5 # Client learning rate

# Client dataset setting
EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] = 2000
EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT'] = 500

# Path to save trained weights and logs
EXPERIMENT_CONFIG['RESULTS_DIRECTORY'] = os.path.join(
    '.', 'results',
    'mobilebert_mlm_stackoverflow_fedavg',
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)

EXPERIMENT_CONFIG['RESULTS_LOG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "logs")
EXPERIMENT_CONFIG['RESULTS_MODEL'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "model")
EXPERIMENT_CONFIG['RESULTS_CONFIG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "config")

In [13]:
# Dump all the configuration into a json file
pathlib.Path(EXPERIMENT_CONFIG['RESULTS_CONFIG']).mkdir(parents=True, exist_ok=True)

with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "config.json"), 'w') as config_file:
    json.dump(EXPERIMENT_CONFIG, config_file, indent=6)

In [14]:
# TFF executor factory settings
# Reference: https://www.tensorflow.org/federated/api_docs/python/tff/backends/native/set_local_execution_context
tff.backends.native.set_local_execution_context(
    num_clients=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
    max_fanout=100,
    clients_per_thread=2,
    server_tf_device=tf_logical_devices_cpu[0],
    client_tf_devices=tf_logical_devices_cpu,
)

## Dataset

### Dataset loader

In [15]:
train_client_data = tff.python.simulation.hdf5_client_data.HDF5ClientData(
    os.path.join('.', 'tff_cache', 'datasets', 'stackoverflow_train.h5'))
# held_out_client_data = tff.python.simulation.hdf5_client_data.HDF5ClientData(
#     os.path.join('.', 'tff_cache', 'datasets', 'stackoverflow_held_out.h5'))
test_client_data = tff.python.simulation.hdf5_client_data.HDF5ClientData(
    os.path.join('.', 'tff_cache', 'datasets', 'stackoverflow_test.h5'))

### Tokenizer

In [16]:
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

In [17]:
# Imitate transformers tokenizer with TF.Text Tokenizer
tokenizer_tf_text, vocab_lookup_table, special_ids_mask_table = \
datasets.preprocessing_for_bert.convert_huggingface_tokenizer(bert_tokenizer)

### Preprocessing

In [18]:
def check_empty_snippet(x):
    return tf.strings.length(x['tokens']) > 0

def tokenizer_and_mask_wrapped(x):

    masked, labels = datasets.preprocessing_for_bert.tokenize_and_mask(tf.reshape(x['tokens'], shape=[1]),
                                                                       max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
                                                                       bert_tokenizer_tf_text=tokenizer_tf_text,
                                                                       vocab_lookup_table=vocab_lookup_table,
                                                                       special_ids_mask_table=special_ids_mask_table,
                                                                       cls_token_id=bert_tokenizer.cls_token_id,
                                                                       sep_token_id=bert_tokenizer.sep_token_id,
                                                                       pad_token_id=bert_tokenizer.pad_token_id,
                                                                       mask_token_id=bert_tokenizer.mask_token_id)

    return (masked, labels)

def preprocess_for_train(train_dataset):
    return (
        train_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly BATCH_SIZE
        # and make the shape **exactly** (BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['BATCH_SIZE'])
        # NOTE: THIS SHOULD BE COMMENTED OUT FOR CENTRALIZED TRAINING
        #.repeat(count=EXPERIMENT_CONFIG['CENTRALIZED_EPOCHS'])
    )
    
def preprocess_for_test(test_dataset):
    return (
        test_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly TEST_BATCH_SIZE
        # and make the shape **exactly** (TEST_BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['TEST_BATCH_SIZE'])
    )

### Training set

In [19]:
train_client_data = train_client_data.preprocess(preprocess_fn=lambda x: x.filter(check_empty_snippet))

In [20]:
train_client_data = train_client_data.preprocess(preprocess_fn=preprocess_for_train)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [21]:
print(train_client_data.element_type_structure)

(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None))


In [22]:
# Since the dataset is pretty large, we randomly select TRAIN_NUM_CLIENT_LIMIT number of clients.
all_train_client_ids = train_client_data.client_ids

random.shuffle(all_train_client_ids)

if EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] > 0:
    selected_train_client_ids = all_train_client_ids[0:EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT']]
else:
    selected_train_client_ids = all_train_client_ids

In [23]:
# Save the list of selected training clients
with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "train_clients_list.json"), 'w') as train_clients_file:
    json.dump(selected_train_client_ids, train_clients_file, indent=6)

In [24]:
train_client_states = {}

# Initialize client states for all clients (selected for the entire simulation)
for i, client_id in enumerate(selected_train_client_ids):
    train_client_states[client_id] = fedavg_client.ClientState(client_serial=i, num_processed=0)

### Test set

In [25]:
test_client_data = test_client_data.preprocess(preprocess_fn=lambda x: x.filter(check_empty_snippet))

In [26]:
test_client_data = test_client_data.preprocess(preprocess_fn=preprocess_for_test)

In [27]:
print(test_client_data.element_type_structure)

(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None))


In [28]:
# Since the dataset is pretty large, we randomly select TEST_NUM_CLIENT_LIMIT number of clients.
all_test_client_ids = test_client_data.client_ids

random.shuffle(all_test_client_ids)

if EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT'] > 0:
    selected_test_client_ids = all_test_client_ids[0:EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT']]
else:
    selected_test_client_ids = all_test_client_ids

In [29]:
# Save the list of selected training clients
with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "test_clients_list.json"), 'w') as test_clients_file:
    json.dump(selected_test_client_ids, test_clients_file, indent=6)

## Model

In [30]:
bert_model = transformers.TFAutoModelForPreTraining.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

All model checkpoint layers were used when initializing TFMobileBertForPreTraining.

All the layers of TFMobileBertForPreTraining were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertForPreTraining for predictions without further training.


In [31]:
print(bert_model.config)

MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "trigram_input": true,
  "true_hidden_size": 128,
  "type_vocab_size": 2,
  "use_bottleneck": true,
  "use_bottleneck_attention": false,
  "vocab_size": 30522
}



In [32]:
# Due to the limitations with Keras subclasses, we can only use the main layer part from pretrained models
# and add output heads by ourselves
bert_keras_converted = utils.convert_huggingface_mlm_to_keras(
    huggingface_model=bert_model,
    max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
)

In [33]:
# Use lists of NumPy arrays to backup pretained weights
bert_pretrained_trainable_weights = []
bert_pretrained_non_trainable_weights = []

for w in bert_keras_converted.trainable_weights:
    bert_pretrained_trainable_weights.append(w.numpy())

for w in bert_keras_converted.non_trainable_weights:
    bert_pretrained_non_trainable_weights.append(w.numpy())

In [34]:
def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""

    loss = utils.MaskedLMCrossEntropy()

    model_wrapped = utils.KerasModelWrapper(
        tf.keras.models.clone_model(bert_keras_converted),
        train_client_data.element_type_structure, loss)

    return model_wrapped

## Training

### Training setups

In [35]:
summary_writer = tf.summary.create_file_writer(EXPERIMENT_CONFIG['RESULTS_LOG'])

In [36]:
def server_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'])

def client_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'])

In [37]:
%%time

iterative_process = fedavg.build_federated_averaging_process(
    model_fn=tff_model_fn,
    model_input_spec=train_client_data.element_type_structure,
    initial_trainable_weights=bert_pretrained_trainable_weights,
    initial_non_trainable_weights=bert_pretrained_non_trainable_weights,
    server_optimizer_fn=server_optimizer_fn, 
    client_optimizer_fn=client_optimizer_fn)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


CPU times: user 2min 16s, sys: 3.94 s, total: 2min 20s
Wall time: 2min 20s


In [38]:
%%time

server_state = iterative_process.initialize()

CPU times: user 20 s, sys: 3.23 s, total: 23.2 s
Wall time: 23 s


In [39]:
metric_eval = tfa.metrics.MeanMetricWrapper(fn=utils.calculate_masked_lm_cross_entropy, name='ce')

In [40]:
# The model for calculating validation loss only
# (This happens outside FedAvg)
model_final = utils.KerasModelWrapper(
    tf.keras.models.clone_model(bert_keras_converted),
    train_client_data.element_type_structure,
    utils.MaskedLMCrossEntropy(),
    tf_device_identifier="/GPU:0")

### Training loop

In [41]:
%%time

with summary_writer.as_default():
    for round_num in range(1, EXPERIMENT_CONFIG['TOTAL_ROUNDS'] + 1):        
        # FedAvg
        print(f'Round {round_num} start!')

        # Training client selection
        sampled_client_serials = np.random.choice(
            len(selected_train_client_ids),
            size=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
            replace=False)

        sampled_train_data = [
            train_client_data.create_tf_dataset_for_client(selected_train_client_ids[client_serial])
            for client_serial in sampled_client_serials
        ]
        
        sampled_client_states = [
            train_client_states[selected_train_client_ids[client_serial]]
            for client_serial in sampled_client_serials
        ]
        
        print("Selected client serials:", sampled_client_serials)

        current_round_start_time = time.time()
        
        server_state, new_client_states, train_loss = iterative_process.next(
            server_state, sampled_client_states, sampled_train_data)
        
        current_round_end_time = time.time()
        
        currnt_round_running_time = current_round_end_time - current_round_start_time

        print(f'Round {round_num} training loss: {train_loss}')
        print(f'Round {round_num} execution time: {currnt_round_running_time}')
        
        # Record the current round's training loss to the log
        tf.summary.scalar('train_loss', train_loss, step=round_num)
        tf.summary.scalar('train_running_time', currnt_round_running_time, step=round_num)
        
        print()
        
        # Update client states
        print("Updating client states.")

        for state in new_client_states:
            train_client_states[selected_train_client_ids[state.client_serial]] = state

        print()
        
        print("Recording client statistics:")
        
        for client_id in selected_train_client_ids:
            state = train_client_states[client_id]
            
            tf.summary.scalar(
                'client_' + str(int(state.client_serial)) + '_num_processed',
                int(state.num_processed), step=round_num)

        print()

        # Evaluation
        if round_num % EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] == 0:
            model_final.from_weights(server_state.model_weights)

            # Test dataset generation for this round
            print("Calculating validation metric:")
            
            # Select test clients
            sampled_test_data = [
                test_client_data.create_tf_dataset_for_client(test_client_id)
                for test_client_id in selected_test_client_ids
            ]

            current_round_validation_start_time = time.time()
            
            current_round_validation_metric = utils.keras_evaluate(
                model_final.keras_model, sampled_test_data, metric_eval, "/GPU:0")
            
            current_round_validation_end_time = time.time()
            
            current_round_validation_runnning_time = current_round_validation_end_time - current_round_validation_start_time

            print(f'Round {round_num} validation metric: {current_round_validation_metric}')
            print(f'Round {round_num} validation time: {current_round_validation_runnning_time}')
            
            # Write down train_metrics to the log
            tf.summary.scalar('validation_metric', current_round_validation_metric, step=round_num)
            tf.summary.scalar('validation_running_time', current_round_validation_runnning_time, step=round_num)
            
        print()

Round 1 start!
Selected client serials: [1504 1797 1327 1890  350  211 1292 1645  997 1034]
Anonymous client 1890 : updated the model with server message.
Anonymous client 1890 : training start.
Anonymous client 1890 : batch 1 , 16 examples processed
Anonymous client 1890 : batch 2 , 32 examples processed
Anonymous client 1890 : batch 3 , 48 examples processed
Anonymous client 1890 : batch 4 , 64 examples processed
Anonymous client 1890 : batch 5 , 80 examples processed
Anonymous client 1890 : batch 6 , 96 examples processed
Anonymous client 1890 : batch 7 , 112 examples processed
Anonymous client 1890 : batch 8 , 128 examples processed
Anonymous client 1890 : batch 9 , 134 examples processed
Anonymous client 1890 : training finished. 134  examples processed, loss: 10.3256588
Anonymous client 997 : updated the model with server message.
Anonymous client 997 : training start.
Anonymous client 997 : batch 1 , 16 examples processed
Anonymous client 997 : batch 2 , 32 examples processed
An

Anonymous client 1292 : batch 37 , 592 examples processed
Anonymous client 211 : batch 15 , 240 examples processed
Anonymous client 1292 : batch 38 , 608 examples processed
Anonymous client 211 : batch 16 , 256 examples processed
Anonymous client 1292 : batch 39 , 624 examples processed
Anonymous client 211 : batch 17 , 272 examples processed
Anonymous client 1292 : batch 40 , 640 examples processed
Anonymous client 211 : batch 18 , 288 examples processed
Anonymous client 1292 : batch 41 , 656 examples processed
Anonymous client 211 : batch 19 , 304 examples processed
Anonymous client 1292 : batch 42 , 672 examples processed
Anonymous client 211 : batch 20 , 320 examples processed
Anonymous client 1292 : batch 43 , 688 examples processed
Anonymous client 211 : batch 21 , 336 examples processed
Anonymous client 1292 : batch 44 , 704 examples processed
Anonymous client 211 : batch 22 , 347 examples processed
Anonymous client 1292 : batch 45 , 720 examples processed
Anonymous client 211 :

AttributeError: 'int' object has no attribute 'numpy'

### Save the trained model

In [42]:
model_final.keras_model.save(EXPERIMENT_CONFIG['RESULTS_MODEL'])

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ./results/mobilebert_mlm_stackoverflow_fedavg/20201128-065644/model/assets


INFO:tensorflow:Assets written to: ./results/mobilebert_mlm_stackoverflow_fedavg/20201128-065644/model/assets
